<a href="https://colab.research.google.com/github/aayushrai/COVID-19-Mask-Recognition/blob/master/Train_model_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')
!unzip "/content/drive/My Drive/type_of_mask_dataset.zip"

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
Archive:  /content/drive/My Drive/type_of_mask_dataset.zip
replace dataset/hankerchief/with-mask-0.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace dataset/hankerchief/with-mask-1.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [1]:
# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten,Conv2D,MaxPooling2D,Activation
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import numpy as np
import os

In [4]:
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
         '/content/dataset',
        target_size=(150, 150),
        batch_size=20,color_mode="grayscale")
validation_generator = test_datagen.flow_from_directory(
         '/content/dataset',
        target_size=(150, 150),
        batch_size=20,color_mode="grayscale")

Found 2669 images belonging to 4 classes.
Found 2669 images belonging to 4 classes.


In [5]:
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150, 150,1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Conv2D(32, (3, 3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(4))
model.add(Activation('sigmoid'))

In [6]:
model.compile(loss="categorical_crossentropy", optimizer="adam",metrics=["accuracy"])

In [7]:
# train the head of the network
print("[INFO] training head...")
H = model.fit(
	train_generator,
	steps_per_epoch=80,
	validation_data=validation_generator,
	validation_steps=80,epochs=50)

[INFO] training head...
Epoch 1/50
80/80 [==============================] - 14s 177ms/step - loss: 1.3308 - accuracy: 0.3650 - val_loss: 1.3534 - val_accuracy: 0.3456
Epoch 2/50
80/80 [==============================] - 14s 171ms/step - loss: 1.1989 - accuracy: 0.4181 - val_loss: 1.3269 - val_accuracy: 0.3256
Epoch 3/50
80/80 [==============================] - 14s 176ms/step - loss: 1.0966 - accuracy: 0.4984 - val_loss: 1.2198 - val_accuracy: 0.4475
Epoch 4/50
80/80 [==============================] - 14s 172ms/step - loss: 1.0661 - accuracy: 0.5125 - val_loss: 1.3944 - val_accuracy: 0.3375
Epoch 5/50
80/80 [==============================] - 14s 175ms/step - loss: 1.0578 - accuracy: 0.5494 - val_loss: 1.5473 - val_accuracy: 0.3131
Epoch 6/50
80/80 [==============================] - 14s 173ms/step - loss: 0.9805 - accuracy: 0.5765 - val_loss: 1.3031 - val_accuracy: 0.4069
Epoch 7/50
80/80 [==============================] - 14s 175ms/step - loss: 0.9159 - accuracy: 0.6149 - val_loss: 1.405

In [8]:
print("[INFO] saving mask detector model...")
model.save("mask_type_keras.h5", save_format="h5")

[INFO] saving mask detector model...
